<a href="https://colab.research.google.com/github/gmoraissc/Projetos_de_Data_Science/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install emoji

In [4]:
!pip install xlsxwriter

In [5]:
# imports
import pandas as pd
import numpy as np
import tweepy, nltk, json, os, csv, xlsxwriter
from pandas import ExcelWriter as ExcelWriter
from pathlib import Path
from datetime import datetime, date
from csv import writer

In [6]:
access_token="590375619-tkJialGWR4R5ZoXJfWsZlx0DoLUfSDlbUi1hLATZ"
access_token_secret="cGjIJU3YM23f1qxH8LvUKd6Tm05zbOBUI9NcCDxLzJ39T"
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAHeVYAEAAAAA9sjLNUPQJgclF6kKG0lPvLTq9nY%3DwprwxmnxRvmXpbU2CbXAoIGogKKEZCRxKwoi6xNW3au27KlCqk'
consumer_key= 'vS7GEC5Cy7HixHoVKNpCR7cFr'
consumer_secret="0cpjFY2tWCwDoWhqOwr2ZBDy6TbpNOtWnQOpU70V9V0Bwu0JNt"

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [8]:
class pesquisas_de_tweets(object):

  def __init__(self):
    pass

  def pesquisar_por_termo(self, query, result_type='recent', count=int(1000)):
    
    global api

    self.query = query
    self.result_type = result_type
    self.count = count

    resultados_da_pesquisa = []

    for status in tweepy.Cursor(api.search,
                                q=self.query,
                                result_type=self.result_type,
                                count=self.count).items():
                                
                                resultados_da_pesquisa.append(status)

    return resultados_da_pesquisa, result_type
 
  def pesquisar_por_usuario(self, participante, result_type='recent', count=int(1000)):
    
    global api, participantes_camarote, participantes_pipoca

    self.participante = participante
    self.result_type = result_type
    self.count = count
    
    tipo_participante = checar_tipo_participante(self.participante)

    resultados_da_pesquisa = []

    for status in tweepy.Cursor(api.user_timeline,
                                screen_name=self.participante,
                                result_type=self.result_type,
                                exclude_replies=False).items(self.count):
                                
                                resultados_da_pesquisa.append(status)

    return resultados_da_pesquisa, result_type, tipo_participante

In [9]:
def criar_dicionario_pesquisas(tweets_status, tipo):
    
  global d1

  popular_dict = {}
  recents_dict = {}

  for each_json_tweet in tweets_status:
    
    _ = json.dumps(each_json_tweet._json)

    if tipo == 'popular':
      
      if d1 not in popular_dict.keys():
        tweet = {d1: [json.loads(_)]}
        popular_dict.update(tweet)
    
      else:
        popular_dict[d1].append(json.loads(_))
    
      return popular_dict, 'popular_dict', tipo

    else:
      
      if d1 not in recents_dict.keys():
        tweet = {d1: [json.loads(_)]}
        recents_dict.update(tweet)

      else:
        recents_dict[d1].append(json.loads(_))
        
      return recents_dict, 'recents_dict', tipo

In [10]:
d1 = date.today().strftime("%d/%m/%Y")

In [22]:
def criar_dataframe(dicionario, tipo_de_pesquisa):
  
  tipo_de_pesquisa = tipo_de_pesquisa

  lista = []

  for i in range(0, len(dicionario[d1])):

        tweet_id = dicionario[d1][i]['id']
        text = dicionario[d1][i]['text']
        favorite_count = dicionario[d1][i]['favorite_count']
        retweet_count = dicionario[d1][i]['retweet_count']
        created_at = dicionario[d1][i]['created_at']
        location = dicionario[d1][i]['user']['location']
        verified = dicionario[d1][i]['user']['verified']
        hashtags = dicionario[d1][i]['entities']['hashtags']
        user_mentions = dicionario[d1][i]['entities']['user_mentions']
        name = dicionario[d1][i]['user']['name']
        screen_name = dicionario[d1][i]['user']['screen_name']

        tipo_participante = checar_tipo_participante(screen_name)

        lista.append({'tweet_id': str(tweet_id),
                      'text': str(text),
                      'favorite_count': int(favorite_count),
                      'retweet_count': int(retweet_count),
                      'created_at': created_at,
                      'location': location,
                      'verified': verified,
                      'hashtags': hashtags,
                      'user_mentions': user_mentions,
                      'name': name,
                      'screen_name': screen_name})
        
        tweet_json_ = pd.DataFrame(lista, columns = 
                                  ['tweet_id', 'text', 
                                    'favorite_count', 'retweet_count', 
                                    'created_at', 'location',
                                    'verified', 'hashtags',
                                    'user_mentions', 'name',
                                    'screen_name'])
        
        return tweet_json_, tipo_participante, tipo_de_pesquisa

In [12]:
def salvar_csvs_no_diretorio(tipo_participante, tipo_pesquisa, dataframe):
  
  dir_csvs = f'/content/drive/MyDrive/01.Instagram profissional/NLP/Sentiment Analysis/BBB22/Twitter/DataFrames'
  data_extracao = d1.replace('/', '')
  file_variable = dir_csvs + "/" + tipo_participante  + "/" + tipo_pesquisa + "/" + data_extracao
  file_name = tipo_participante[0:3] + "_" + tipo_pesquisa[0:3] + "_" + data_extracao + '.csv'
  file_path = file_variable + "/" + file_name

  if not os.path.exists(file_variable):
    os.makedirs(file_variable)
    dataframe.to_csv(file_path, index=False)

  else:
    with open(file_path, 'a') as f_object:
      writer_object = writer(f_object, delimiter=',')
      linhas = []
      
      for row in range(0, len(dataframe)):
        linhas.append([row for row in dataframe.iloc[row]])
        writer_object.writerow(linhas[row])

      f_object.close()

In [14]:
def checar_tipo_participante(usuario):
  if usuario in participantes_camarote:
    return 'Camarote'
  elif usuario in participantes_pipoca:
    return 'Pipoca'
  else:
    return 'Publico'

In [84]:
termos_de_pesquisa_BBB = ('BBB OR BBB22 OR BIG BROTHER BRASIL OR BIG BROTHER BRASIL 2022 OR OR #REDEBBB OR BIGBROTHER OR #BIGBROTHER OR #BBB OR #BBB22')

termos_de_pesquisa_participantes = {'Arthur': ['Arthur', 'Artur', 'Aguiar', 'Arthur Aguiar', '\u2747\ufe0f', '#TeamAguiar', '#TeamArthurAguiar'],
                                    'Naiara Azevedo': ['Naiara', 'Azevedo', '1f4b8', '#TeamNaiara', 'Nai', 'cantora'],
                                    'Pedro Scooby': ['Pedro', 'Scooby', '1f30a', '#TeamScooby', '#TimeScooby', '1f499'],
                                    'Brunna Gonçalves': ['Brunna', 'Gonçalves', '1f984', '#BBBRUNNA', 'BBBrunna', ''],
                                    'Paulo André': ['Paulo', 'André', '1f3c1', '#TeamPauloAndré', 'TeamPauloAndre'],
                                    'Maria': ['Maria', '1f40d	', '#TEAMMARIA', 'TIME MARIA', 'MARICONAS', ''],
                                    'Jade Picon': ['Jade', 'Picon', '1f32a\ufe0f', 'furacão', '#TeamJade', 'Picão', 'KdJade', 'Picões', 'Furacao'],
                                    'Douglas Silva': ['Douglas', 'Silva', '1f3b2', 'dado', 'dadinho',
                                    '#TeamDouglasSilva', '1f44a\U0001f3ff'],
                                    'Linn da Quebrada': ['Linn', 'Quebrada', '1f9dc\u200d\u2640\ufe0f', '1f9dc\U0001f3ff\u200d\u2640\ufe0f',
                                    '1f9dc\U0001f3fb\u200d\u2640\ufe0f', '1f9dc\U0001f3fe\u200d\u2640\ufe0f', '1f9dc\U0001f3fc\u200d\u2640\ufe0f',
                                    '1f9dc\U0001f3fd\u200d\u2640\ufe0f', '1f9dc', '1f9dc\U0001f3ff', '1f9dc\U0001f3fb', '1f9dc\U0001f3fe',
                                    '1f9dc\U0001f3fc', '1f9dc\U0001f3fd', '#TeamLinn', '#linndonas', '#LinnDonas'],
                                    'Tiago Abravanel': ['Tiago', 'Abravanel', '1f43b', '#TeamAbrava', '#TeamAbravanel', '1f9f8'],
                                    'Laís Caldas': ['Laís', 'Caldas', '1f462', '#TeamLais', '#TeamLaís', 'Time Lais', '1f49a'],
                                    'Luciano Estevan': ['Luciano', 'Estevan', '1f981', '#TeamLucianoEstevan', 'Lu', '1f346'],
                                    'Jessilane': ['Jessilane', '1f9ec', 'Jessi', '#TeamJessi', '#TimeJessi', '1f49c', 'Charmanders', ''],
                                    'Eliezer': ['Eliezer'],
                                    'Eslovênia Marques': ['Eslovênia', 'Eslováquia', 'Marques', 'Eslô', '1f1f8\U0001f1ee',
                                                          'Time Eslo', 'Team Eslo', '#TimeEslô', '#TeamEslô',
                                                          'Eslovenia'],
                                    'Bábara Heck': ['Bárbara', 'Heck', '1f980', '#TeamBá', 'TeamBá', 'TimeBá',
                                                    'TimeBa', 'TeamBa', 'TeamBah', 'BBBah', 'Bá', 'Ba', '1f9a6'],
                                    'Rodrigo Mussi': ['Rodrigo Mussi', 'Mussi', 'Rodrigo', 'TEAM MUSSI', '#TeamMussi',
                                                      '#TimeMussi', '1f977\U0001f3ff', '1f977', '1f977\U0001f3fb',
                                                      '1f977\U0001f3fe', '1f977\U0001f3fc', '1f977\U0001f3fd', 'ninja', 'ninjas',
                                                      'TEAM NINJA', ''],
                                    'Natália Deodato': ['Natália', 'Deodato', '#TeamNaty', '#TimeNaty', 'Naty', 'vitilindos',
                                                        '1f483', '1f483\U0001f3ff', '1f483\U0001f3fb',
                                                        '1f483\U0001f3fe', '1f483\U0001f3fc',
                                                        '1f483\U0001f3fd'],
                                    'Vinicius': ['Vinicius', 'TeamVyni', '#TimeVyni', '#TeamVyni', '1f4a1'],
                                    'Lucas Bissoli': ['Lucas', 'Bissoli', '#TimeBissoli', '#TeamBissoli', 
                                                      '1f3c4\u200d\u2642\ufe0f', '1f3c4\U0001f3ff\u200d\u2642\ufe0f', 
                                                      '1f3c4\U0001f3fb\u200d\u2642\ufe0f', '1f3c4\U0001f3fe\u200d\u2642\ufe0f',
                                                      '1f3c4\U0001f3fc\u200d\u2642\ufe0f', '1f3c4\U0001f3fd\u200d\u2642\ufe0f',
                                                      '1f3c4', '1f3c4\U0001f3ff', '1f3c4\U0001f3fb', '1f3c4\U0001f3fe',
                                                      '1f3c4\U0001f3fc', '1f3c4\U0001f3fd', '1f3c4\u200d\u2640\ufe0f',
                                                      '1f3c4\U0001f3ff\u200d\u2640\ufe0f', '1f3c4\U0001f3fb\u200d\u2640\ufe0f',
                                                      '1f3c4\U0001f3fe\u200d\u2640\ufe0f', '1f3c4\U0001f3fc\u200d\u2640\ufe0f',
                                                      '1f3c4\U0001f3fd\u200d\u2640\ufe0f']}
                                    
dicionario_participantes = {'@Aguiarthur': 'Arthur Aguiar', 
                            '@Naiarazevedo': 'Naiara Azevedo',
                            '@PedroScooby': 'Pedro Scooby',
                            '@brunnagoncalves': 'Bruna Gonçalves',
                            '@iampauloandre': 'Paulo André',
                            '@eumaria': 'Maria',
                            '@jadepicon': 'Jade Picon',
                            '@Silva_DG': 'Douglas Silva',
                            '@linndaquebrada': 'Linn da Quebrada',
                            '@TiagoAbravanel': 'Tiago Abravanel',
                            '@Dra_laiscaldass': 'Laís Caldas',
                            '@LucianoEstevan': 'Luciano Estevan',
                            '@a_jessilane': 'Jessilane',
                            '@Eliezerbbb22': 'Eliezer',
                            '@eslomarques': 'Eslovênia Maques',
                            '@bbaheck': 'Bábara Heck',
                            '@oficialmussi': 'Rodrigo Mussi',
                            '@oficial_deodato': 'Natália Deodato',
                            '@vyniof': 'Vinicius',
                            '@LucasBissoli_': 'Lucas Bissoli'}

participantes_camarote = ['Aguiarthur', 'Naiarazevedo', 'PedroScooby', 'brunnagoncalves', 
                          'iampauloandre', 'eumaria', 'jadepicon', 'Silva_DG', 'linndaquebrada', 'TiagoAbravanel']

participantes_pipoca = ['Dra_laiscaldass', 'LucianoEstevan', 'a_jessilane', 'Eliezerbbb22', 'eslomarques', 
                        'bbaheck', 'oficialmussi', 'oficial_deodato', 'vyniof', 'LucasBissoli_']

participantes_camarote_twitter = ['@Aguiarthur', '@Naiarazevedo', '@PedroScooby', '@brunnagoncalves', 
                          '@iampauloandre', '@eumaria', '@jadepicon', '@Silva_DG', '@linndaquebrada', '@TiagoAbravanel']

participantes_pipoca_twitter = ['@Dra_laiscaldass', '@LucianoEstevan', '@a_jessilane', '@Eliezerbbb22', '@eslomarques', 
                        '@bbaheck', '@oficialmussi', '@oficial_deodato', '@vyniof', '@LucasBissoli_']

participantes_total = []
participantes_total.extend(participantes_pipoca)
participantes_total.extend(participantes_camarote)

In [20]:
tweets, tipo_de_pesquisa, tipo_participante = pesquisas_de_tweets().pesquisar_por_usuario(participante='@Naiarazevedo',
                                                         result_type='recent')
dicionario, nome_do_dicionario, tipo_de_pesquisa = criar_dicionario_pesquisas(tweets, tipo_de_pesquisa)
dataframe, tipo_participante, tipo_de_pesquisa = criar_dataframe(dicionario, tipo_de_pesquisa)
salvar_csvs_no_diretorio(tipo_participante, tipo_de_pesquisa, dataframe)

In [118]:
def pesquisar_infos_usuario(screen_name):
  
  global api

  informacoes_hist_participante = []

  user_results = api.get_user(screen_name)

  extracao = datetime.now()
  participante = user_results.name
  twitter = user_results.screen_name
  seguidores = user_results.followers_count
  seguindo = user_results.friends_count
  numero_de_listados = user_results.listed_count
  n_tweets_favoritados = user_results.favourites_count
  num_de_tweets_totais = user_results.statuses_count
  verificado = user_results.verified

  informacoes_hist_participante.append({'participante': str(participante),
                                        'twitter': str(twitter),
                                        'n_seguidores': int(seguidores),
                                        'n_seguindo': int(seguindo),
                                        'n_de_listados': int(numero_de_listados),
                                        'n_tweets_favoritados': int(n_tweets_favoritados),
                                        'num_de_tweets_totais': int(num_de_tweets_totais),
                                        'verificado': bool(verificado),
                                        'data': extracao})
  
  return informacoes_hist_participante

def dataframe_dos_participantes(lista):

  dados_participantes = pd.DataFrame(lista, columns = 
                                  ['participante', 'twitter', 'n_seguidores', 
                                   'n_seguindo','n_de_listados', 'n_tweets_favoritados',
                                   'num_de_tweets_totais', 'verificado', 'data'])
  
  return dados_participantes

def historico_infos(participantes):
  
  historico_info_participantes = []

  for participante in participantes:
    _ = pesquisar_infos_usuario(participante)
    historico_info_participantes.extend(_)
  
  return historico_info_participantes

In [117]:
dataframe_dos_participantes(historico_infos(participantes_total))

participante  ...                       data
0            Laís Caldas 👢  ... 2022-01-19 23:50:20.226652
1        Luciano Estevan 🦁  ... 2022-01-19 23:50:20.389675
2        Jessilane Alves 🧬  ... 2022-01-19 23:50:20.527115
3                  Eliezer  ... 2022-01-19 23:50:20.691258
4          Eslô Marques 🇸🇮  ... 2022-01-19 23:50:20.860861
5           Bárbara Heck 🦀  ... 2022-01-19 23:50:21.026896
6          Rodrigo Mussi 🥷  ... 2022-01-19 23:50:21.229609
7       Natália Deodato 💃🏾  ... 2022-01-19 23:50:21.422954
8                   Vyni 💡  ... 2022-01-19 23:50:21.557971
9      Lucas Bissoli 🏄🏼‍♂️  ... 2022-01-19 23:50:21.690869
10         Arthur Aguiar ✨  ... 2022-01-19 23:50:21.823028
11        Naiara Azevedo 💸  ... 2022-01-19 23:50:21.958453
12          Pedro Scooby 🌊  ... 2022-01-19 23:50:22.105442
13      Brunna Gonçalves 🦄  ... 2022-01-19 23:50:22.240012
14           Paulo André 🏁  ... 2022-01-19 23:50:22.371864
15                 MARIA 🐍  ... 2022-01-19 23:50:22.504650
16            jade picon 🌪  ... 2022-01-19 23:50:22.675383
17         Douglas Silva 🎲  ... 2022-01-19 23:50:22.822953
18  Linn da Quebrada 🧜🏽‍♀️  ... 2022-01-19 23:50:22.983163
19       Tiago Abravanel 🧸  ... 2022-01-19 23:50:23.115677

[20 rows x 9 columns]

In [109]:
for participante in participantes_total:
  df = dataframe_dos_participantes(pesquisar_infos_usuario(participante)) 
df.head()

ValueError: ignored

In [ ]:
tweet_json_recents.join(tweet_json_popular, on='tweet_id')

ValueError: ignored

In [ ]:
calendar = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
  
tweet_json['Week_day'] = tweet_json['created_at'].str.slice(0,3)
tweet_json['Month_name'] = tweet_json['created_at'].str.slice(4,7)
tweet_json['Month'] = tweet_json['Month_name'].map(calendar)
tweet_json['Day_number'] = tweet_json['created_at'].str.slice(8,10)
tweet_json['Tweeted_at_time'] = tweet_json['created_at'].str.slice(11,19)
tweet_json['Year'] = tweet_json['created_at'].str.slice(26,30)

NameError: ignored

In [ ]:
tweet_json['Data'] = str(str(tweet_json['Day_number']) + '-' + str(tweet_json['Month']) + '-' + str(tweet_json['Year']))

In [ ]:
tweet_json['Data'][0]

'0     15\n1     14\n2     14\n3     15\n4     14\n5     14\n6     15\n7     15\n8     14\n9     14\n10    15\n11    14\n12    14\n13    14\nName: Day_number, dtype: object-0     1\n1     1\n2     1\n3     1\n4     1\n5     1\n6     1\n7     1\n8     1\n9     1\n10    1\n11    1\n12    1\n13    1\nName: Month, dtype: int64-0     2022\n1     2022\n2     2022\n3     2022\n4     2022\n5     2022\n6     2022\n7     2022\n8     2022\n9     2022\n10    2022\n11    2022\n12    2022\n13    2022\nName: Year, dtype: object'

In [ ]:
# em média, quantos seguidores ganha após um tweet positivo? e quantos perde após um tweet negativo?
# modelo de classificação baseado na popularidade do participante
# prob de ser eliminado no paredão de acordo com o atual nível de popularidade

In [ ]:
pipocas_part.head(3)

tweet_id  ...      screen_name
0  1483265947175632897  ...  Dra_laiscaldass
1  1483265820528680961  ...  Dra_laiscaldass
2  1483265480945287169  ...  Dra_laiscaldass

[3 rows x 13 columns]

In [ ]:
word_tokenize(pipocas_part['text'])

NameError: ignored

In [ ]:
entrada_e_saida = {'Participante': 'a',
                   'Entrada': data,
                   'Eliminacao': data}